# T1070.005 - Indicator Removal on Host: Network Share Connection Removal
Adversaries may remove share connections that are no longer useful in order to clean up traces of their operation. Windows shared drive and [SMB/Windows Admin Shares](https://attack.mitre.org/techniques/T1021/002) connections can be removed when no longer needed. [Net](https://attack.mitre.org/software/S0039) is an example utility that can be used to remove network share connections with the <code>net use \\system\share /delete</code> command. (Citation: Technet Net Use)

## Atomic Tests

### Atomic Test #1 - Add Network Share
Add a Network Share utilizing the command_prompt

**Supported Platforms:** windows
#### Attack Commands: Run with `command_prompt`
```command_prompt
net use c: \\test\share
net share test=\\test\share /REMARK:"test share" /CACHE:No
```

In [ ]:
Invoke-AtomicTest T1070.005 -TestNumbers 1

### Atomic Test #2 - Remove Network Share
Removes a Network Share utilizing the command_prompt

**Supported Platforms:** windows
#### Attack Commands: Run with `command_prompt`
```command_prompt
net share \\test\share /delete
```

In [ ]:
Invoke-AtomicTest T1070.005 -TestNumbers 2

### Atomic Test #3 - Remove Network Share PowerShell
Removes a Network Share utilizing PowerShell

**Supported Platforms:** windows
#### Attack Commands: Run with `powershell`
```powershell
Remove-SmbShare -Name \\test\share
Remove-FileShare -Name \\test\share
```

In [ ]:
Invoke-AtomicTest T1070.005 -TestNumbers 3

### Atomic Test #4 - Disable Administrative Share Creation at Startup
Administrative shares are hidden network shares created by Microsoft’s Windows NT operating systems that grant system administrators 
remote access to every disk volume on a network-connected system. These shares are automatically created at started unless they have been
purposefully disabled as is done in this Atomic test. As Microsoft puts it, "Missing administrative shares typically 
indicate that the computer in question has been compromised by malicious software."
https://threatpost.com/conti-ransomware-gang-has-full-log4shell-attack-chain/177173/

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_LOCAL_MACHINE\SYSTEM\CurrentControlSet\Services\LanmanServer\Parameters" /v AutoShareServer /t REG_DWORD /d 0 /f
reg add "HKEY_LOCAL_MACHINE\SYSTEM\CurrentControlSet\Services\LanmanServer\Parameters" /v AutoShareWks /t REG_DWORD /d 0 /f
```

In [ ]:
Invoke-AtomicTest T1070.005 -TestNumbers 4

### Atomic Test #5 - Remove Administrative Shares
Administrative shares are hidden network shares created by Microsoft’s Windows NT operating systems that grant system administrators 
remote access to every disk volume on a network-connected system. As Microsoft puts it, “Missing administrative shares typically 
indicate that the computer in question has been compromised by malicious software.
https://threatpost.com/conti-ransomware-gang-has-full-log4shell-attack-chain/177173/

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
for %i in (C$ IPC$ ADMIN$) do net share %i /delete```

In [ ]:
Invoke-AtomicTest T1070.005 -TestNumbers 5

## Detection
Network share connections may be common depending on how an network environment is used. Monitor command-line invocation of <code>net use</code> commands associated with establishing and removing remote shares over SMB, including following best practices for detection of [Windows Admin Shares](https://attack.mitre.org/techniques/T1077). SMB traffic between systems may also be captured and decoded to look for related network share session and file transfer activity. Windows authentication logs are also useful in determining when authenticated network shares are established and by which account, and can be used to correlate network share activity to other events to investigate potentially malicious activity.